In [3]:
import sys

from pathlib import Path
main_path = Path().absolute().parent
data_path = main_path / 'data'
setting_path = main_path / 'setting_files'

sys.path.append(str(main_path / 'src'))

import torch
import yaml
import json
import pytorch_lightning as pl
from pytorch_lightning import seed_everything

from nlu_models import NLUModel
from nlu_utils import NLUDataModule

with (setting_path / 'train_settings.yml').open('r') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

data_module_settings = settings['data_module']
model_settings = settings['model']
trainer_settings = settings['trainer']

data_module = NLUDataModule(
    train_path=data_path / data_module_settings['train_file'], 
    valid_path=data_path / data_module_settings['valid_file'],
    test_path=data_path / data_module_settings['test_file'],
    labels_path=data_path / data_module_settings['labels_file'],
    batch_size=data_module_settings['batch_size'], 
    max_len=data_module_settings['max_len'],
    num_workers=data_module_settings['num_workers'],
    seed=settings['seed']
)

if model_settings.get('weight_file') is not None:
    with (data_path / model_settings['weight_file']).open('r', encoding='utf-8') as file:
        weight_dict = json.load(file)
else:
    weight_dict = None

if trainer_settings['deterministic']:
    seed_everything(seed=settings['seed'], workers=True)
deterministic = trainer_settings['deterministic']

trainer = pl.Trainer(
    gpus=trainer_settings['n_gpus'], 
    max_epochs=trainer_settings['n_epochs'], 
    num_sanity_val_steps=trainer_settings['num_sanity_val_steps'],
    log_every_n_steps=trainer_settings['log_every_n_steps'],
    deterministic=deterministic,
)

# trainer.test(ckpt_path='best', datamodule=data_module)

# checkpoint_path = str(main_path / 'checkpoints' / 'nlu_simple' / 'best_model.ckpt')

# seed_everything(seed=settings['seed'])
# trainer = pl.Trainer(
#     gpus=trainer_settings['n_gpus'], 
#     max_epochs=trainer_settings['n_epochs'], 
#     num_sanity_val_steps=trainer_settings['num_sanity_val_steps'],
#     deterministic=True,
# )
# # checkpoint = torch.load(checkpoint_path)
# model = NLUModel.load_from_checkpoint(checkpoint_path)
# # model.load_state_dict(checkpoint['state_dict'])
# model.eval()
# print()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [4]:
from nlu_utils import NLUTokenizer

tokenizer = NLUTokenizer()

In [21]:
model_idx = 8
model_dict = dict(enumerate([
    ('ce_l7_tk3_0', 'epoch=11-step=167-val_loss=1.74.ckpt'),     # 0
    ('ce_l7_tk3_1', 'epoch=8-step=125-val_loss=1.57.ckpt'),      # 1
    ('ce_l7_tk3_3', 'epoch=6-step=97-val_loss=1.82.ckpt'),       # 2
    ('focal_l7_tk3_0', 'epoch=17-step=251-val_loss=0.43.ckpt'),  # 3
    ('focal_l7_tk3_1', 'epoch=19-step=279-val_loss=0.35.ckpt'),  # 4
    ('focal_l7_tk3_3', 'epoch=18-step=265-val_loss=0.44.ckpt'),  # 5
    ('focal2_l7_tk3_0', 'epoch=18-step=265-val_loss=0.25.ckpt'), # 6
    ('focal2_l7_tk3_1', 'epoch=19-step=279-val_loss=0.17.ckpt'), # 7
    ('focal2_l7_tk3_3', 'epoch=18-step=265-val_loss=0.30.ckpt'), # 8
]))
s = model_dict[model_idx]
print(s)
checkpoint_path = str(main_path / 'logs' / 'nlu_simple' / s[0] / 'checkpoints' / s[1])
model = NLUModel.load_from_checkpoint(checkpoint_path)
data = {"text": "current assets"}
text = data['text']
bert_encodes = tokenizer(
    text, 
    add_special_tokens=True, 
    truncation=True, 
    max_length=256,
    return_tensors='pt'
)
o = model(**bert_encodes)
o

('focal2_l7_tk3_3', 'epoch=18-step=265-val_loss=0.30.ckpt')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

{'tags': tensor([[  7.2178, -11.1062, -18.4225,   5.0288,   6.1428,   4.4894,   5.1955,
          -11.9663,  -9.8212, -11.4087,  -6.8118],
         [  7.2178, -11.1062, -18.4225,   5.0288,   6.1428,   4.4894,   5.1955,
          -11.9663,  -9.8212, -11.4087,  -6.8118],
         [  7.2178, -11.1062, -18.4225,   5.0288,   6.1428,   4.4894,   5.1955,
          -11.9663,  -9.8212, -11.4087,  -6.8118],
         [  7.2178, -11.1062, -18.4225,   5.0288,   6.1428,   4.4894,   5.1955,
          -11.9663,  -9.8212, -11.4087,  -6.8118]], grad_fn=<ViewBackward0>),
 'intent': tensor([[  4.2094,   2.9006, -24.7791,  -2.1780]], grad_fn=<AddmmBackward0>)}